In [ ]:
!nvidia-smi

Wed Jul 10 11:42:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
image_height=64
image_width=64
batch_size=32

In [ ]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/My Drive/LS24-Machine Learning/homer_bart',
    labels='inferred',
    label_mode='int',
    class_names=['Bart','Homer'],
    color_mode='rgb',
    image_size=(image_height,image_width),
    shuffle=True,
    seed=123,
)
#Splitting dataset in a 9:1 ratio
dataset_size = tf.data.experimental.cardinality(ds_train).numpy()
train_size = int(0.9 * dataset_size)
test_size = dataset_size - train_size

train_dataset = ds_train.take(train_size)
test_dataset = ds_train.skip(train_size)

Found 269 files belonging to 2 classes.


In [ ]:
model = keras.Sequential([
    keras.Input(shape=(64,64,3)),
    layers.Flatten(),
    layers.Dense(128,activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(64,activation='relu',kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(2,activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 dense (Dense)               (None, 128)               1572992   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1585538 (6.05 MB)
Trainable params: 1585538 (6.05 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model = keras.models.load_model()

In [ ]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.Adam(learning_rate=0.0001),
    metrics = ['accuracy']
)
model.fit(train_dataset,epochs=20,validation_data=test_dataset, verbose=2 )
loss, acc = model.evaluate(test_dataset,verbose=2)
print(f"The accuracy of this model is {acc}" )

Epoch 1/20
8/8 - 59s - loss: 38.9375 - accuracy: 0.5039 - val_loss: 19.0411 - val_accuracy: 0.3846 - 59s/epoch - 7s/step
Epoch 2/20
8/8 - 2s - loss: 16.6869 - accuracy: 0.6133 - val_loss: 9.3349 - val_accuracy: 0.6154 - 2s/epoch - 303ms/step
Epoch 3/20
8/8 - 2s - loss: 10.6394 - accuracy: 0.6602 - val_loss: 11.7545 - val_accuracy: 0.6154 - 2s/epoch - 221ms/step
Epoch 4/20
8/8 - 2s - loss: 8.4164 - accuracy: 0.7227 - val_loss: 7.0421 - val_accuracy: 0.6154 - 2s/epoch - 212ms/step
Epoch 5/20
8/8 - 2s - loss: 7.1174 - accuracy: 0.7383 - val_loss: 5.3366 - val_accuracy: 0.8462 - 2s/epoch - 215ms/step
Epoch 6/20
8/8 - 2s - loss: 6.0518 - accuracy: 0.8086 - val_loss: 7.5542 - val_accuracy: 0.7692 - 2s/epoch - 219ms/step
Epoch 7/20
8/8 - 2s - loss: 7.2090 - accuracy: 0.7422 - val_loss: 4.1665 - val_accuracy: 0.9231 - 2s/epoch - 271ms/step
Epoch 8/20
8/8 - 2s - loss: 7.5299 - accuracy: 0.7109 - val_loss: 4.3699 - val_accuracy: 0.8462 - 2s/epoch - 300ms/step
Epoch 9/20
8/8 - 2s - loss: 5.5752 -

In [ ]:
model.save()